# Setup

In [2]:
from google.colab import files
import os

# 1. Hiện nút upload file
print("Hãy chọn file kaggle.json từ máy tính của bạn...")
uploaded = files.upload()

# 2. Tạo thư mục ẩn .kaggle (nếu chưa có)
!mkdir -p ~/.kaggle

# 3. Di chuyển file vào đúng chỗ
!cp kaggle.json ~/.kaggle/

# 4. Phân quyền (QUAN TRỌNG: nếu không làm bước này Kaggle sẽ báo lỗi bảo mật)
!chmod 600 ~/.kaggle/kaggle.json

print("Đã cài đặt xong! Giờ bạn có thể dùng lệnh kaggle.")

Hãy chọn file kaggle.json từ máy tính của bạn...


Saving kaggle.json to kaggle.json
Đã cài đặt xong! Giờ bạn có thể dùng lệnh kaggle.


In [3]:
import os

if not os.path.exists("MABe-mouse-behavior-detection.zip"):
    print("Downloading dataset...")
    !kaggle competitions download -c MABe-mouse-behavior-detection
else:
    print("Zip already exists, skip download.")

if not os.path.exists("./input"):
    print("Unzipping files...")
    !unzip -q MABe-mouse-behavior-detection.zip -d ./input
else:
    print("Input folder already exists, skip unzip.")

100% 2.62G/2.63G [00:02<00:00, 848MB/s] 
100% 2.63G/2.63G [00:02<00:00, 1.38GB/s]
Unzipping files...


In [4]:
!pip install -U lightgbm cupy-cuda12x

In [5]:
!pip install -q polars xgboost scikit-learn catboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 13.6 MB/s eta 0:00:00


In [6]:
%%writefile metric.py
"""F Beta customized for the data format of the MABe challenge."""

import json

from collections import defaultdict

import pandas as pd
import polars as pl


class HostVisibleError(Exception):
    pass


def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()  # ty: ignore
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            # Since the labels are sparse, we can't evaluate prediction keys not in the active labels.
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue

            new_frames = set(range(row['start_frame'], row['stop_frame']))
            # Ignore truly redundant predictions.
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                # A single agent can have multiple targets per frame (ex: evading all other mice) but only one action per target per frame.
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)


def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    """
    Doctests:
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 0, 'stop_frame': 10}, # Wrong action
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.500000000000'

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.250000000000'

    >>> # Overlapping solution events, one prediction matching both.
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 10, 'stop_frame': 20, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 20},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 30, 'stop_frame': 40, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 40},
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.6666666666666666
    """
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    # Need to align based on video IDs as we can't rely on the row IDs for handling public/private splits.
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    """
    F1 score for the MABe Challenge
    """
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

Writing metric.py


# Config

In [7]:
import datetime
import gc
import itertools
import json
import re
import sys
import time
import traceback
from collections import defaultdict
from pathlib import Path

import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.auto import tqdm

from metric import score

In [8]:
# const
# INPUT_DIR = Path("/kaggle/input/MABe-mouse-behavior-detection")
# WORKING_DIR = Path("/kaggle/working")

INPUT_DIR = Path("/content/input")
WORKING_DIR = Path("/content/working")


TRAIN_TRACKING_DIR = INPUT_DIR / "train_tracking"
TRAIN_ANNOTATION_DIR = INPUT_DIR / "train_annotation"
TEST_TRACKING_DIR = INPUT_DIR / "test_tracking"


INDEX_COLS = [
    "video_id",
    "agent_mouse_id",
    "target_mouse_id",
    "video_frame",
]

BODY_PARTS = [
    "ear_left",
    "ear_right",
    "nose",
    "neck",
    "body_center",
    "lateral_left",
    "lateral_right",
    "hip_left",
    "hip_right",
    "tail_base",
    "tail_tip",
]

SELF_BEHAVIORS = [
    "biteobject",
    "climb",
    "dig",
    "exploreobject",
    "freeze",
    "genitalgroom",
    "huddle",
    "rear",
    "rest",
    "run",
    "selfgroom",
]

PAIR_BEHAVIORS = [
    "allogroom",
    "approach",
    "attack",
    "attemptmount",
    "avoid",
    "chase",
    "chaseattack",
    "defend",
    "disengage",
    "dominance",
    "dominancegroom",
    "dominancemount",
    "ejaculate",
    "escape",
    "flinch",
    "follow",
    "intromit",
    "mount",
    "reciprocalsniff",
    "shepherd",
    "sniff",
    "sniffbody",
    "sniffface",
    "sniffgenital",
    "submit",
    "tussle",
]

In [9]:
# read data
train_dataframe = pl.read_csv(INPUT_DIR / "train.csv")

# Preprocessing

## Behavior Labels

In [10]:
# preprocess behavior labels
train_behavior_dataframe = (
    train_dataframe
    .filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
    )
    .explode("behaviors_labeled_list")
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled_element").str.split(",").list[0].str.replace_all("'", "").alias("agent"),
        pl.col("behaviors_labeled_element").str.split(",").list[1].str.replace_all("'", "").alias("target"),
        pl.col("behaviors_labeled_element").str.split(",").list[2].str.replace_all("'", "").alias("behavior"),
    )
)

train_self_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(SELF_BEHAVIORS))
train_pair_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(PAIR_BEHAVIORS))

## Self


*   Khoảng cách giữa các bộ phận cơ thể của agent (cm) - BODY PARTS

*   Vận tốc ước tính của các bộ phận (cm/s)
Tính vận tốc ước tính của ear_left, ear_right, tail_base trong các khoảng thời gian 500, 1000, 2000, 3000 ms.

*   Độ duỗi (nose-tail base / earleft-right)

*   Body angle (nose-body center vs body center-tail)





In [11]:
%%writefile self_features.py

def make_self_features(
    metadata: dict,
    tracking: pl.DataFrame,
) -> pl.DataFrame:
    # tạo để đỡ phải truy cập nhiều lần
    fps = metadata["frames_per_second"]
    pix_per_cm = metadata["pix_per_cm_approx"]

    # Helper
    def get_window(period_ms):
        return max(1, int(round(period_ms * fps / 1000.0)))


    def body_parts_distance(body_part_1, body_part_2):
        # Khoảng cách giữa các bộ phận cơ thể của agent (cm)
        assert body_part_1 in BODY_PARTS
        assert body_part_2 in BODY_PARTS
        return (
            (pl.col(f"agent_x_{body_part_1}") - pl.col(f"agent_x_{body_part_2}")).pow(2)
            + (pl.col(f"agent_y_{body_part_1}") - pl.col(f"agent_y_{body_part_2}")).pow(2)
        ).sqrt() / metadata["pix_per_cm_approx"]

    # thêm chức năng switch để chuyển giữa mean vafd std
    def body_part_speed(body_part, period_ms, stat="mean"):
        assert body_part in BODY_PARTS
        # Tốc độ ước tính của bộ phận (cm/s)
        raw_speed = (
            ((pl.col(f"agent_x_{body_part}").diff()).pow(2) + (pl.col(f"agent_y_{body_part}").diff()).pow(2)).sqrt()
            / pix_per_cm * fps
        )
        w = get_window(period_ms)
        if stat == "mean":
            return raw_speed.rolling_mean(window_size=w, center=True, min_samples=1).fill_null(0.0)
        elif stat == "std":
            return raw_speed.rolling_std(window_size=w, center=True, min_samples=1).fill_null(0.0)
        return raw_speed

    def elongation():
        # Độ giãn dài
        d1 = body_parts_distance("nose", "tail_base")
        d2 = body_parts_distance("ear_left", "ear_right")
        return d1 / (d2 + 1e-06)

    def body_angle():
        # Góc cơ thể (deg)
        v1x = pl.col("agent_x_nose") - pl.col("agent_x_body_center")
        v1y = pl.col("agent_y_nose") - pl.col("agent_y_body_center")
        v2x = pl.col("agent_x_tail_base") - pl.col("agent_x_body_center")
        v2y = pl.col("agent_y_tail_base") - pl.col("agent_y_body_center")
        return (v1x * v2x + v1y * v2y) / ((v1x.pow(2) + v1y.pow(2)).sqrt() * (v2x.pow(2) + v2y.pow(2)).sqrt() + 1e-06)

    # [MỚI] Hàm tính Grooming Decouple
    def grooming_decouple():
        # Tốc độ tức thời của Mũi
        s_nose = (((pl.col("agent_x_nose").diff()).pow(2) + (pl.col("agent_y_nose").diff()).pow(2)).sqrt() / pix_per_cm * fps)
        # Tốc độ tức thời của Thân
        s_body = (((pl.col("agent_x_body_center").diff()).pow(2) + (pl.col("agent_y_body_center").diff()).pow(2)).sqrt() / pix_per_cm * fps)

        # Ratio: Mũi / (Thân + 0.5) -> Median 500ms
        w = get_window(500)
        ratio = (s_nose / (s_body + 0.5)).clip(0.0, 10.0)
        return ratio.rolling_median(window_size=w, center=True, min_samples=1).fill_null(0.0)
    # [MỚI] Hàm tính Nose Radial Jitter
    def nose_radial_jitter():
        # Khoảng cách Mũi - Thân
        dist = body_parts_distance("nose", "body_center")
        # Std trong 500ms
        w = get_window(500)
        return dist.rolling_std(window_size=w, center=True, min_samples=1).fill_null(0.0)
    # [MỚI] Hàm tính Vận tốc góc
    def angular_velocity():
        vec_x = pl.col("agent_x_nose") - pl.col("agent_x_body_center")
        vec_y = pl.col("agent_y_nose") - pl.col("agent_y_body_center")
        angle = pl.arctan2(vec_y, vec_x)
        # Diff góc * FPS = Rad/s -> Smooth 300ms
        w = get_window(300)
        return (angle.diff().abs() * fps).rolling_mean(window_size=w, center=True, min_samples=1).fill_null(0.0)



    n_mice = (
        (metadata["mouse1_strain"] is not None)
        + (metadata["mouse2_strain"] is not None)
        + (metadata["mouse3_strain"] is not None)
        + (metadata["mouse4_strain"] is not None)
    )
    start_frame = tracking.select(pl.col("video_frame").min()).item()
    end_frame = tracking.select(pl.col("video_frame").max()).item()

    result = []

    pivot = tracking.pivot(
        on=["bodypart"],
        index=["video_frame", "mouse_id"],
        values=["x", "y"],
    ).sort(["mouse_id", "video_frame"])
    pivot_trackings = {mouse_id: pivot.filter(pl.col("mouse_id") == mouse_id) for mouse_id in range(1, n_mice + 1)}

    for agent_mouse_id in range(1, n_mice + 1):
        result_element = pl.DataFrame(
            {
                "video_id": metadata["video_id"],
                "agent_mouse_id": agent_mouse_id,
                "target_mouse_id": -1,
                "video_frame": pl.arange(start_frame, end_frame + 1, eager=True),
            },
            schema={
                "video_id": pl.Int32,
                "agent_mouse_id": pl.Int8,
                "target_mouse_id": pl.Int8,
                "video_frame": pl.Int32,
            },
        )

        pivot = pivot_trackings[agent_mouse_id].select(
            pl.col("video_frame"),
            pl.exclude("video_frame").name.prefix("agent_"),
        )
        columns = pivot.columns
        pivot = pivot.with_columns(
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_x_{bp}") for bp in BODY_PARTS if f"agent_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_y_{bp}") for bp in BODY_PARTS if f"agent_y_{bp}" not in columns],
        )

        features = pivot.with_columns(
            pl.lit(agent_mouse_id).alias("agent_mouse_id"),
            pl.lit(-1).alias("target_mouse_id"),
        ).select(
            pl.col("video_frame"),
            pl.col("agent_mouse_id"),
            pl.col("target_mouse_id"),
            *[
                body_parts_distance(body_part_1, body_part_2).alias(f"aa__{body_part_1}__{body_part_2}__distance")
                for body_part_1, body_part_2 in itertools.combinations(BODY_PARTS, 2)
            ],
            *[
                body_part_speed(body_part, period_ms).alias(f"agent__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            # THÊM: body_center speed (Run/Walk)
            *[
                body_part_speed("body_center", ms, stat="mean").alias(
                    f"agent__body_center__speed_{ms}ms"
                )
                for ms in [500, 1000, 2000]
            ],
            # THÊM: nose speed (Groom/Sniff)
            *[
                body_part_speed("nose", ms, stat="mean").alias(
                    f"agent__nose__speed_{ms}ms"
                )
                for ms in [500, 1000]
            ],
            elongation().alias("agent__elongation"),
            body_angle().alias("agent__body_angle"),
            # các feature mới thêm
            grooming_decouple().alias("agent__groom_decouple"),
            nose_radial_jitter().alias("agent__groom_nose_jitter"),
            angular_velocity().alias("agent__angular_velocity"),
        )

        result_element = result_element.join(
            features,
            on=["video_frame", "agent_mouse_id", "target_mouse_id"],
            how="left",
        )
        result.append(result_element)

    return pl.concat(result, how="vertical")

Writing self_features.py


## Pair

*   Khoảng cách giữa các bộ phận cơ thể của agent–target (cm)  - BODY PARTS

*   Vận tốc ước tính của các bộ phận của agent và target (cm/s)
Tính vận tốc ước tính của ear_left, ear_right, tail_base trong các khoảng thời gian 500, 1000, 2000, 3000 ms cho cả agent và target.

*   Độ duỗi của agent và target

*   Góc cơ thể của agent và target


In [12]:
%%writefile pair_features.py

def make_pair_features(
    metadata: dict,
    tracking: pl.DataFrame,
) -> pl.DataFrame:
    def body_parts_distance(agent_or_target_1, body_part_1, agent_or_target_2, body_part_2):
        # Khoảng cách giữa các bộ phận cơ thể của agent-target (cm)
        assert agent_or_target_1 == "agent" or agent_or_target_1 == "target"
        assert agent_or_target_2 == "agent" or agent_or_target_2 == "target"
        assert body_part_1 in BODY_PARTS
        assert body_part_2 in BODY_PARTS
        return (
            (pl.col(f"{agent_or_target_1}_x_{body_part_1}") - pl.col(f"{agent_or_target_2}_x_{body_part_2}")).pow(2)
            + (pl.col(f"{agent_or_target_1}_y_{body_part_1}") - pl.col(f"{agent_or_target_2}_y_{body_part_2}")).pow(2)
        ).sqrt() / metadata["pix_per_cm_approx"]

    def body_part_speed(agent_or_target, body_part, period_ms):
        # Tốc độ ước tính của bộ phận (cm/s)
        assert agent_or_target == "agent" or agent_or_target == "target"
        assert body_part in BODY_PARTS
        window_frames = max(1, int(round(period_ms * metadata["frames_per_second"] / 1000.0)))
        return (
            (
                (pl.col(f"{agent_or_target}_x_{body_part}").diff()).pow(2)
                + (pl.col(f"{agent_or_target}_y_{body_part}").diff()).pow(2)
            ).sqrt()
            / metadata["pix_per_cm_approx"]
            * metadata["frames_per_second"]
        ).rolling_mean(window_size=window_frames, center=True, min_samples=1).fill_null(0.0)

    def elongation(agent_or_target):
        # Độ giãn dài (cm)
        assert agent_or_target == "agent" or agent_or_target == "target"
        d1 = body_parts_distance(agent_or_target, "nose", agent_or_target, "tail_base")
        d2 = body_parts_distance(agent_or_target, "ear_left", agent_or_target, "ear_right")
        return d1 / (d2 + 1e-06)

    def body_angle(agent_or_target):
        # Cosine of body angle (alignment)
        assert agent_or_target == "agent" or agent_or_target == "target"
        v1x = pl.col(f"{agent_or_target}_x_nose") - pl.col(f"{agent_or_target}_x_body_center")
        v1y = pl.col(f"{agent_or_target}_y_nose") - pl.col(f"{agent_or_target}_y_body_center")
        v2x = pl.col(f"{agent_or_target}_x_tail_base") - pl.col(f"{agent_or_target}_x_body_center")
        v2y = pl.col(f"{agent_or_target}_y_tail_base") - pl.col(f"{agent_or_target}_y_body_center")
        return (v1x * v2x + v1y * v2y) / ((v1x.pow(2) + v1y.pow(2)).sqrt() * (v2x.pow(2) + v2y.pow(2)).sqrt() + 1e-06)

    def body_center_distance_rolling_agg(agg, period_ms):
        # Đặc trưng tổng hợp khoảng cách trung tâm cơ thể di chuyển
        assert agg in ["mean", "std", "var", "min", "max"] # Hàm tổng hợp
        expr = body_parts_distance("agent", "body_center", "target", "body_center")
        window_frames = max(1, int(round(period_ms * metadata["frames_per_second"] / 1000.0)))

        if agg == "mean":
            return expr.rolling_mean(window_size=window_frames, center=True, min_samples=1)
        elif agg == "std":
            return expr.rolling_std(window_size=window_frames, center=True, min_samples=1)
        elif agg == "var":
            return expr.rolling_var(window_size=window_frames, center=True, min_samples=1)
        elif agg == "min":
            return expr.rolling_min(window_size=window_frames, center=True, min_samples=1)
        elif agg == "max":
            return expr.rolling_max(window_size=window_frames, center=True, min_samples=1)
        else:
            raise ValueError()

    # Add new feature
    fps  = metadata["frames_per_second"]
    def body_center_distance():
        # Khoảng cách tâm thân agent–target (cm)
        return body_parts_distance("agent", "body_center", "target", "body_center")
    def body_center_radial_velocity(period_ms=300):
        """
        Vận tốc hướng tâm (cm/s):
        - < 0: lại gần (approach/chase)
        - > 0: xa ra (avoid/escape)
        """
        dist = body_center_distance()
        window_frames = max(1, int(round(period_ms * fps / 1000.0)))
        return (dist.diff()*fps).rolling_mean(
            window_size=window_frames,
            center=True,
            min_samples=1,
        ).fill_null(0.0)
    def relative_speed(period_ms=500):
        """
        Chênh lệch tốc độ thân:
        > 0: agent nhanh hơn
        < 0: target nhanh hơn
        """
        return (
            body_part_speed("agent", "body_center", period_ms)
            - body_part_speed("target", "body_center", period_ms)
        )
    def facing_score(agent_role, target_role, period_ms=500):
        """
        Cosine giữa:
        - hướng body_center→nose của agent
        - vector agent_body_center → target_body_center
        """
        vec_ag_x = pl.col(f"{agent_role}_x_nose") - pl.col(f"{agent_role}_x_body_center")
        vec_ag_y = pl.col(f"{agent_role}_y_nose") - pl.col(f"{agent_role}_y_body_center")

        vec_to_tg_x = pl.col(f"{target_role}_x_body_center") - pl.col(f"{agent_role}_x_body_center")
        vec_to_tg_y = pl.col(f"{target_role}_y_body_center") - pl.col(f"{agent_role}_y_body_center")

        dot = vec_ag_x * vec_to_tg_x + vec_ag_y * vec_to_tg_y
        mag_ag = (vec_ag_x.pow(2) + vec_ag_y.pow(2)).sqrt()
        mag_to = (vec_to_tg_x.pow(2) + vec_to_tg_y.pow(2)).sqrt()

        cos_val = (dot / (mag_ag * mag_to + 1e-6)).clip(-1.0, 1.0)

        window_frames = max(1, int(round(period_ms * fps / 1000.0)))
        return cos_val.rolling_mean(
            window_size=window_frames,
            center=True,
            min_samples=1,
        ).fill_null(0.0)


    n_mice = (
        (metadata["mouse1_strain"] is not None)
        + (metadata["mouse2_strain"] is not None)
        + (metadata["mouse3_strain"] is not None)
        + (metadata["mouse4_strain"] is not None)
    )
    start_frame = tracking.select(pl.col("video_frame").min()).item()
    end_frame = tracking.select(pl.col("video_frame").max()).item()

    result = []

    pivot = tracking.pivot(
        on=["bodypart"],
        index=["video_frame", "mouse_id"],
        values=["x", "y"],
    ).sort(["mouse_id", "video_frame"])
    pivot_trackings = {mouse_id: pivot.filter(pl.col("mouse_id") == mouse_id) for mouse_id in range(1, n_mice + 1)}

    for agent_mouse_id, target_mouse_id in itertools.permutations(range(1, n_mice + 1), 2):
        result_element = pl.DataFrame(
            {
                "video_id": metadata["video_id"],
                "agent_mouse_id": agent_mouse_id,
                "target_mouse_id": target_mouse_id,
                "video_frame": pl.arange(start_frame, end_frame + 1, eager=True),
            },
            schema={
                "video_id": pl.Int32,
                "agent_mouse_id": pl.Int8,
                "target_mouse_id": pl.Int8,
                "video_frame": pl.Int32,
            },
        )

        merged_pivot = (
            pivot_trackings[agent_mouse_id]
            .select(
                pl.col("video_frame"),
                pl.exclude("video_frame").name.prefix("agent_"),
            )
            .join(
                pivot_trackings[target_mouse_id].select(
                    pl.col("video_frame"),
                    pl.exclude("video_frame").name.prefix("target_"),
                ),
                on="video_frame",
                how="inner",
            )
        )
        columns = merged_pivot.columns
        merged_pivot = merged_pivot.with_columns(
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_x_{bp}") for bp in BODY_PARTS if f"agent_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_y_{bp}") for bp in BODY_PARTS if f"agent_y_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"target_x_{bp}") for bp in BODY_PARTS if f"target_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"target_y_{bp}") for bp in BODY_PARTS if f"target_y_{bp}" not in columns],
        )

        features = merged_pivot.with_columns(
            pl.lit(agent_mouse_id).alias("agent_mouse_id"),
            pl.lit(target_mouse_id).alias("target_mouse_id"),
        ).select(
            pl.col("video_frame"),
            pl.col("agent_mouse_id"),
            pl.col("target_mouse_id"),
            *[
                body_parts_distance("agent", agent_body_part, "target", target_body_part).alias(
                    f"at__{agent_body_part}__{target_body_part}__distance"
                )
                for agent_body_part, target_body_part in itertools.product(BODY_PARTS, repeat=2)
            ],
            *[
                body_part_speed("agent", body_part, period_ms).alias(f"agent__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            *[
                body_part_speed("target", body_part, period_ms).alias(f"target__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            elongation("agent").alias("agent__elongation"),
            elongation("target").alias("target__elongation"),
            body_angle("agent").alias("agent__body_angle"),
            body_angle("target").alias("target__body_angle"),

            # 1) Speed body_center của agent & target (locomotion tương đối)
            body_part_speed("agent", "body_center", 500).alias("agent__body_center__speed_500ms"),
            body_part_speed("target", "body_center", 500).alias("target__body_center__speed_500ms"),

            # 2) Khoảng cách tâm thân + rolling mean
            body_center_distance().alias("at__body_center__distance"),
            body_center_distance_rolling_agg("mean", 500).alias("at__body_center__distance_mean_500ms"),
            body_center_distance_rolling_agg("mean", 1000).alias("at__body_center__distance_mean_1000ms"),

            # 3) Động học khoảng cách & speed tương đối
            body_center_radial_velocity().alias("at__body_center__radial_velocity"),
            relative_speed().alias("pair__body_center_speed_diff_500ms"),

            # 4) Facing score: agent nhìn target & target nhìn agent
            facing_score("agent", "target").alias("pair__agent_facing_score"),
            facing_score("target", "agent").alias("pair__target_facing_score"),
        )

        result_element = result_element.join(
            features,
            on=["video_frame", "agent_mouse_id", "target_mouse_id"],
            how="left",
        )
        result.append(result_element)

    return pl.concat(result, how="vertical")

Writing pair_features.py


In [13]:
%run -i self_features.py
%run -i pair_features.py

def process_video(row):
    """Process a single video to extract self and pair features."""
    lab_id = row["lab_id"]
    video_id = row["video_id"]

    tracking_path = TRAIN_TRACKING_DIR / f"{lab_id}/{video_id}.parquet"
    tracking = pl.read_parquet(tracking_path)

    self_features = make_self_features(metadata=row, tracking=tracking)
    pair_features = make_pair_features(metadata=row, tracking=tracking)

    self_features.write_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
    pair_features.write_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")

    return video_id


# make data
(WORKING_DIR / "self_features").mkdir(exist_ok=True, parents=True)
(WORKING_DIR / "pair_features").mkdir(exist_ok=True, parents=True)

rows = list(train_dataframe.filter(pl.col("behaviors_labeled").is_not_null()).rows(named=True))
results = joblib.Parallel(n_jobs=-1, verbose=5)(joblib.delayed(process_video)(row) for row in rows)

print(f"Processed {len(results)} videos successfully")

del rows, results
gc.collect()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.4min


Processed 848 videos successfully


[Parallel(n_jobs=-1)]: Done 848 out of 848 | elapsed:  1.9min finished


27

# Training

In [14]:
SEEDS = [42, 43, 44]

In [15]:
def tune_threshold(oof_action, y_action):
    thresholds = np.arange(0, 1.005, 0.005)
    scores = [f1_score(y_action, (oof_action >= th), zero_division=0) for th in thresholds]
    best_idx = np.argmax(scores)
    return thresholds[best_idx]

*   XGB for perlab, per behavior
*   Cross validation 3 Fold



In [16]:
def train_validate(lab_id: str, behavior: str, indices: pl.DataFrame, features: pl.DataFrame, labels: pl.Series, seed: int = 42):
    # Tạo đường dẫn thư mục để lưu kết quả
    result_dir = WORKING_DIR / "results" / lab_id / behavior / f"seed_{seed}"
    # Tạo thư mục nếu không tồn tại (bao gồm cả thư mục cha)
    result_dir.mkdir(exist_ok=True, parents=True)

    # Xử lý trường hợp tổng nhãn là 0 (không có mẫu dương nào)
    if labels.sum() == 0:
        # Lưu điểm F1 là 0
        with open(result_dir / "f1.txt", "w") as f:
            f.write("0.0\n")
        # Tạo DataFrame kết quả với tất cả các giá trị dự đoán là 0
        oof_prediction_dataframe = indices.with_columns(
            pl.Series("fold", [-1] * len(labels), dtype=pl.Int8),  # Số fold (-1 nghĩa là không sử dụng)
            pl.Series("prediction", [0.0] * len(labels), dtype=pl.Float32),  # Xác suất dự đoán
            pl.Series("predicted_label", [0] * len(labels), dtype=pl.Int8),  # Nhãn dự đoán
        )
        # Lưu kết quả dưới dạng parquet
        oof_prediction_dataframe.write_parquet(result_dir / "oof_predictions.parquet")
        return 0.0

    # Khởi tạo mảng để lưu kết quả dự đoán Out-of-Fold
    folds = np.ones(len(labels), dtype=np.int8) * -1  # Số fold mà mỗi mẫu thuộc về
    oof_predictions = np.zeros(len(labels), dtype=np.float32)  # Xác suất dự đoán
    oof_prediction_labels = np.zeros(len(labels), dtype=np.int8)  # Nhãn dự đoán (0 hoặc 1)

    # Thực hiện phân tích chéo nhóm phân tầng 3 fold
    # StratifiedGroupKFold giữ phân bố nhãn và đảm bảo cùng một nhóm (video_id) không bị chia thành nhiều fold
    cv = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=seed)

    for fold, (train_idx, valid_idx) in enumerate(
        cv.split(
            X=features,
            y=labels,
            groups=indices.get_column("video_id"),
        )
    ):
        # Tạo thư mục để lưu kết quả cho mỗi fold
        result_dir_fold = result_dir / f"fold_{fold}"
        result_dir_fold.mkdir(exist_ok=True, parents=True)

        # Chia thành dữ liệu huấn luyện và xác thực
        X_train = features[train_idx]  # Đặc trưng huấn luyện
        y_train = labels[train_idx]  # Nhãn huấn luyện
        X_valid = features[valid_idx]  # Đặc trưng xác thực
        y_valid = labels[valid_idx]  # Nhãn xác thực

        # Tính trọng số để xử lý mất cân bằng lớp
        # Số mẫu âm / Số mẫu dương = Trọng số cho mẫu dương
        scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

        # Đặt các tham số XGBoost
        params = {
            "objective": "binary:logistic",  # Vấn đề phân loại nhị phân
            "eval_metric": "logloss",  # Chỉ số đánh giá: mất mát logarit
            "device": "cuda",  # Thiết bị sử dụng
            "tree_method": "hist",  # Thuật toán dựa trên histogram nhanh
            "learning_rate": 0.05,  # Tốc độ học
            "max_depth": 6,  # Độ sâu tối đa của cây
            "min_child_weight": 5,  # Trọng số tối thiểu của nút con
            "subsample": 0.8,  # Tỷ lệ mẫu sử dụng cho mỗi cây
            "colsample_bytree": 0.8,  # Tỷ lệ đặc trưng sử dụng cho mỗi cây
            "scale_pos_weight": scale_pos_weight,  # Trọng số của mẫu dương
            "max_bin": 64,  # Số bin của histogram
            "seed": seed,  # Seed ngẫu nhiên
        }

        # Tạo ma trận dữ liệu cho XGBoost (dữ liệu huấn luyện là ma trận lượng tử hóa, dữ liệu xác thực là ma trận thông thường)
        dtrain = xgb.QuantileDMatrix(X_train, label=y_train, feature_names=features.columns, max_bin=64)
        dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_names=features.columns)

        # Từ điển để lưu kết quả đánh giá
        evals_result = {}

        # Đặt callback dừng sớm
        # Dừng huấn luyện nếu mất mát logarit của dữ liệu xác thực không cải thiện trong 10 vòng
        early_stopping_callback = xgb.callback.EarlyStopping(
            rounds=10,  # Số vòng liên tiếp không cải thiện
            metric_name="logloss",  # Chỉ số cần giám sát
            data_name="valid",  # Tập dữ liệu cần giám sát
            maximize=False,  # Chỉ số càng nhỏ càng tốt
            save_best=True,  # Lưu mô hình tốt nhất
        )

        # Thực hiện huấn luyện mô hình
        model = xgb.train(
            params,  # Tham số siêu
            dtrain=dtrain,  # Dữ liệu huấn luyện
            num_boost_round=250,  # Số vòng tăng cường tối đa
            evals=[(dtrain, "train"), (dvalid, "valid")],  # Tập dữ liệu để đánh giá
            callbacks=[early_stopping_callback],  # Callback
            evals_result=evals_result,  # Nơi lưu kết quả đánh giá
            verbose_eval=0,  # Tần suất ghi log (0 là không ghi)
        )

        # Thực hiện dự đoán trên dữ liệu xác thực (lấy giá trị xác suất)
        fold_predictions = model.predict(dvalid)

        # Điều chỉnh ngưỡng tối ưu để tối đa hóa điểm F1
        threshold = tune_threshold(fold_predictions, y_valid)

        # Lưu kết quả dự đoán Out-of-Fold
        folds[valid_idx] = fold  # Số fold
        oof_predictions[valid_idx] = fold_predictions  # Xác suất dự đoán
        oof_prediction_labels[valid_idx] = (fold_predictions >= threshold).astype(np.int8)  # Nhị phân hóa bằng ngưỡng

        # Lưu kết quả của fold này
        # Lưu mô hình đã huấn luyện
        model.save_model(result_dir_fold / "model.json")
        # Lưu ngưỡng tối ưu
        with open(result_dir_fold / "threshold.txt", "w") as f:
            f.write(f"{threshold}\n")

        # Vẽ biểu đồ mức độ quan trọng của đặc trưng (top 20, theo gain)
        xgb.plot_importance(model, max_num_features=20, importance_type="gain", values_format="{v:.2f}")
        plt.tight_layout()
        plt.savefig(result_dir_fold / "feature_importance.png")
        plt.close()

        # Vẽ biểu đồ đường cong học (diễn biến mất mát logarit)
        lgb.plot_metric(evals_result, metric="logloss")
        plt.tight_layout()
        plt.savefig(result_dir_fold / "metric.png")
        plt.close()

        # Giải phóng bộ nhớ
        gc.collect()

    # Tổng hợp kết quả dự đoán của tất cả các fold vào một DataFrame
    oof_prediction_dataframe = indices.with_columns(
        pl.Series("fold", folds, dtype=pl.Int8),  # Số fold
        pl.Series("prediction", oof_predictions, dtype=pl.Float32),  # Xác suất dự đoán
        pl.Series("predicted_label", oof_prediction_labels, dtype=pl.Int8),  # Nhãn dự đoán
    )

    # Tính điểm F1 tổng thể
    f1 = f1_score(labels, oof_prediction_labels, zero_division=0)
    # Lưu điểm F1 vào tệp
    with open(result_dir / "f1.txt", "w") as f:
        f.write(f"{f1}\n")

    # Lưu DataFrame kết quả dự đoán
    oof_prediction_dataframe.write_parquet(result_dir / "oof_predictions.parquet")

    # Trả về điểm F1
    return f1

##Self

In [ ]:
groups = train_self_behavior_dataframe.group_by("lab_id", "behavior", maintain_order=True)
total_groups = len(list(groups))
start_time = time.perf_counter()

for idx, ((lab_id, behavior), group) in tqdm(enumerate(groups), total=total_groups):
    if idx == 0:
        tqdm.write(
            f"|{'LAB':^25}|{'BEHAVIOR':^15}|{'SAMPLES':^10}|{'POSITIVE':^10}|{'FEATURES':^10}|{'F1':^10}|{'ELAPSED TIME':^15}|",
            end="\n",
        )

    tqdm.write(f"|{lab_id:^25}|{behavior:^15}|", end="")
    index_list = []
    feature_list = []
    label_list = []

    for row in group.rows(named=True):
        video_id = row["video_id"]
        agent = row["agent"]

        agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))

        data = pl.scan_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet").filter(
            (pl.col("agent_mouse_id") == agent_mouse_id)
        )
        index = data.select(INDEX_COLS).collect(engine="streaming")
        feature = data.select(pl.exclude(INDEX_COLS)).collect(engine="streaming")

        # read annotation
        annotation_path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        if annotation_path.exists():
            annotation = (
                pl.scan_parquet(annotation_path)
                .filter((pl.col("action") == behavior) & (pl.col("agent_id") == agent_mouse_id))
                .collect()
            )
        else:
            annotation = pl.DataFrame(
                schema={
                    "agent_id": pl.Int8,
                    "target_id": pl.Int8,
                    "action": str,
                    "start_frame": pl.Int16,
                    "stop_frame": pl.Int16,
                }
            )

        label_frames = set()
        for annotation_row in annotation.rows(named=True):
            label_frames.update(range(annotation_row["start_frame"], annotation_row["stop_frame"]))
        label = index.select(pl.col("video_frame").is_in(label_frames).cast(pl.Int8).alias("label"))

        if label.get_column("label").sum() == 0:
            continue

        index_list.append(index)
        feature_list.append(feature)
        label_list.append(label.get_column("label"))

    if not index_list:
        elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
        tqdm.write(f"{0:>10,}|{0:>10,}|{0:>10,}|{'-':>10}|{str(elapsed_time):>15}|", end="\n")
        continue

    indices = pl.concat(index_list, how="vertical")
    features = pl.concat(feature_list, how="vertical")
    labels = pl.concat(label_list, how="vertical")

    del index_list, feature_list, label_list
    gc.collect()

    tqdm.write(f"{len(indices):>10,}|{labels.sum():>10,}|{len(features.columns):>10,}|", end="")

    f1_list = []
    for seed in SEEDS:
        f1 = train_validate(lab_id, behavior, indices, features, labels, seed=seed)
        f1_list.append(f1)
    mean_f1 = float(np.mean(f1_list))
    tqdm.write(f"{mean_f1:>10.2f}|", end="")

    elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
    tqdm.write(f"{str(elapsed_time):>15}|", end="\n")

    gc.collect()

  0%|          | 0/27 [00:00<?, ?it/s]

|           LAB           |   BEHAVIOR    | SAMPLES  | POSITIVE | FEATURES |    F1    | ELAPSED TIME  |
|     AdaptableSnail      |     rear      |   660,348|    85,313|        77|      0.64|        0:01:24|
|         CRIM13          |     rear      |   179,132|    12,042|        77|      0.34|        0:01:57|
|         CRIM13          |   selfgroom   |   205,533|    14,472|        77|      0.37|        0:02:32|
|      CalMS21_task1      | genitalgroom  |   102,445|     6,270|        77|      0.65|        0:02:57|
|       ElegantMink       |     rear      |         0|         0|         0|         -|        0:02:57|
|       ElegantMink       |   selfgroom   |         0|         0|         0|         -|        0:02:57|
|       GroovyShrew       |     rear      |   899,280|    50,768|        77|      0.53|        0:04:36|
|       GroovyShrew       |     rest      |   530,886|    87,573|        77|      0.69|        0:05:36|
|       GroovyShrew       |   selfgroom   |   877,773|    22,893

##Pair

In [ ]:
groups = train_pair_behavior_dataframe.group_by("lab_id", "behavior", maintain_order=True)
total_groups = len(list(groups))
start_time = time.perf_counter()

for idx, ((lab_id, behavior), group) in tqdm(enumerate(groups), total=total_groups):
    if idx == 0:
        tqdm.write(
            f"|{'LAB':^25}|{'BEHAVIOR':^15}|{'SAMPLES':^10}|{'POSITIVE':^10}|{'FEATURES':^10}|{'F1':^10}|{'ELAPSED TIME':^15}|",
            end="\n",
        )

    tqdm.write(f"|{lab_id:^25}|{behavior:^15}|", end="")
    index_list = []
    feature_list = []
    label_list = []

    for row in group.rows(named=True):
        video_id = row["video_id"]
        agent = row["agent"]
        target = row["target"]

        agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))
        target_mouse_id = int(re.search(r"mouse(\d+)", target).group(1))

        data = pl.scan_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet").filter(
            (pl.col("agent_mouse_id") == agent_mouse_id) & (pl.col("target_mouse_id") == target_mouse_id)
        )
        index = data.select(INDEX_COLS).collect(engine="streaming")
        feature = data.select(pl.exclude(INDEX_COLS)).collect(engine="streaming")

        # read annotation
        annotation_path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        if annotation_path.exists():
            annotation = (
                pl.scan_parquet(annotation_path)
                .filter(
                    (pl.col("action") == behavior)
                    & (pl.col("agent_id") == agent_mouse_id)
                    & (pl.col("target_id") == target_mouse_id)
                )
                .collect()
            )
        else:
            annotation = pl.DataFrame(
                schema={
                    "agent_id": pl.Int8,
                    "target_id": pl.Int8,
                    "action": str,
                    "start_frame": pl.Int16,
                    "stop_frame": pl.Int16,
                }
            )

        label_frames = set()
        for annotation_row in annotation.rows(named=True):
            label_frames.update(range(annotation_row["start_frame"], annotation_row["stop_frame"]))
        label = index.select(pl.col("video_frame").is_in(label_frames).cast(pl.Int8).alias("label"))

        if label.get_column("label").sum() == 0:
            continue

        index_list.append(index)
        feature_list.append(feature)
        label_list.append(label.get_column("label"))

    if not index_list:
        elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
        tqdm.write(f"{0:>10,}|{0:>10,}|{0:>10,}|{'-':>10}|{str(elapsed_time):>15}|", end="\n")
        continue

    indices = pl.concat(index_list, how="vertical")
    features = pl.concat(feature_list, how="vertical")
    labels = pl.concat(label_list, how="vertical")

    del index_list, feature_list, label_list
    gc.collect()

    tqdm.write(f"{len(indices):>10,}|{labels.sum():>10,}|{len(features.columns):>10,}|", end="")

    f1_list = []
    for seed in SEEDS:
        f1 = train_validate(lab_id, behavior, indices, features, labels, seed=seed)
        f1_list.append(f1)
    mean_f1 = float(np.mean(f1_list))
    tqdm.write(f"{mean_f1:>10.2f}|", end="")

    elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
    tqdm.write(f"{str(elapsed_time):>15}|", end="\n")

    gc.collect()

In [ ]:
%%writefile robustify.py

def robustify(submission: pl.DataFrame, dataset: pl.DataFrame, train_test: str = "train"):
    traintest_directory = INPUT_DIR / f"{train_test}_tracking"

    old_submission = submission.clone()
    submission = submission.filter(pl.col("start_frame") < pl.col("stop_frame"))
    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")

    old_submission = submission.clone()
    group_list = []
    for _, group in submission.group_by("video_id", "agent_id", "target_id"):
        group = group.sort("start_frame")
        mask = np.ones(len(group), dtype=bool)
        last_stop_frame = 0
        for i, row in enumerate(group.rows(named=True)):
            if row["start_frame"] < last_stop_frame:
                mask[i] = False
            else:
                last_stop_frame = row["stop_frame"]
        group_list.append(group.filter(pl.Series("mask", mask)))

    submission = pl.concat(group_list)

    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")

    # ========= 💡 3. MERGE SMALL GAPS GIỮA 2 EVENT CÙNG ACTION =========
    MAX_GAP_FRAMES = 1

    merged_groups = []
    before_merge = len(submission)

    # Group theo (video, agent, target), DUYỆT THEO THỨ TỰ GLOBAL
    for _, group in submission.group_by(["video_id", "agent_id", "target_id"]):
        group = group.sort("start_frame")
        rows = list(group.rows(named=True))
        if not rows:
            continue

        current = dict(rows[0])
        merged = []

        for row in rows[1:]:
            # Chỉ merge nếu CÙNG action và gap nhỏ
            gap = row["start_frame"] - current["stop_frame"] - 1
            if (row["action"] == current["action"]) and (gap <= MAX_GAP_FRAMES):
                if row["stop_frame"] > current["stop_frame"]:
                    current["stop_frame"] = row["stop_frame"]
            else:
                merged.append(current)
                current = dict(row)

        merged.append(current)
        merged_groups.append(pl.DataFrame(merged, schema=submission.schema))

    if merged_groups:
        submission = pl.concat(merged_groups)

    after_merge = len(submission)
    if after_merge != before_merge:
        print(f"INFO: Merged small gaps, events: {before_merge} -> {after_merge}")
    # =========  THÊM BƯỚC LỌC EVENT QUÁ NGẮN Ở ĐÂY =========
    MIN_LEN_FRAMES = 3
    SHORT_OK = ["flinch"]

    submission = submission.with_columns(
        (pl.col("stop_frame") - pl.col("start_frame")).alias("length")
    )

    before = len(submission)

    submission = submission.filter(
        pl.when(pl.col("action").is_in(SHORT_OK))
          .then(pl.col("length") >= 1)                 # các action “nhấp nháy” không lọc theo min_len
          .otherwise(pl.col("length") >= MIN_LEN_FRAMES)  # còn lại phải dài >= 3 frame
    )

    submission = submission.drop("length")
    after = len(submission)

    if after != before:
        print(f"INFO: Dropped {before - after} short events (action-dependent)")

    # =========================================================


    s_list = []
    for row in dataset.rows(named=True):
        lab_id = row["lab_id"]
        video_id = row["video_id"]
        if row["behaviors_labeled"] is None:
            continue

        if video_id in submission.get_column("video_id").to_list():
            continue

        if isinstance(row["behaviors_labeled"], str):
            continue

        print(f"Video {video_id} has no predictions.")

        path = traintest_directory / f"/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)

        vid_behaviors = json.loads(row["behaviors_labeled"])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(",") for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=["agent", "target", "action"])

        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1

        for (agent, target), actions in vid_behaviors.groupby(["agent", "target"]):
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, action_row in enumerate(actions.itertuples(index=False)):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)
                s_list.append((video_id, agent, target, action_row["action"], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat(
            [
                submission,
                pd.DataFrame(s_list, columns=["video_id", "agent_id", "target_id", "action", "start_frame", "stop_frame"]),
            ]
        )
        print("ERROR: Filled empty videos")

    return submission

## Tổng hợp các giá trị dự đoán trên dữ liệu kiểm chứng

In [ ]:
# Danh sách để lưu kết quả dự đoán Out-of-Fold của từng nhóm
group_oof_predictions: list[pl.DataFrame] = []

# Nhóm dữ liệu theo lab_id, video_id, agent, target
# Dùng list(...) để dùng lại groups + có total cho tqdm
groups = list(
    train_behavior_dataframe.group_by(
        "lab_id", "video_id", "agent", "target",
        maintain_order=True,
    )
)

for (lab_id, video_id, agent, target), group in tqdm(groups, total=len(groups)):
    # Trích xuất ID chuột từ agent (chủ thể hành động), ví dụ: "mouse1" → 1
    agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))

    # Trích xuất ID chuột từ target (đối tượng hành động)
    # Nếu là "self" thì -1, ngược lại lấy ID chuột
    target_mouse_id = -1 if target == "self" else int(re.search(r"mouse(\d+)", target).group(1))

    # Danh sách để lưu score cho từng behavior trong nhóm này
    prediction_dataframe_list: list[pl.DataFrame] = []

    # Lấy danh sách behavior duy nhất trong group
    unique_behaviors = group.get_column("behavior").unique().to_list()

    # Xử lý từng behavior trong nhóm
    for behavior in unique_behaviors:
        # ===== Ensemble OOF qua nhiều seed cho behavior này =====
        pred_dfs_per_seed: list[pl.DataFrame] = []

        for seed in SEEDS:
            # Đường dẫn OOF cho seed này
            oof_path = (
                WORKING_DIR
                / "results"
                / lab_id
                / behavior
                / f"seed_{seed}"
                / "oof_predictions.parquet"
            )

            if not oof_path.exists():
                continue

            # Đọc OOF của seed, lọc đúng video/agent/target
            df_seed = (
                pl.scan_parquet(oof_path)
                .filter(
                    (pl.col("video_id") == video_id)
                    & (pl.col("agent_mouse_id") == agent_mouse_id)
                    & (pl.col("target_mouse_id") == target_mouse_id)
                )
                .select(
                    *INDEX_COLS,
                    # Score seed = prob * predicted_label
                    # (dưới threshold = 0, trên threshold = prob)
                    (pl.col("prediction") * pl.col("predicted_label")).alias(f"{behavior}_seed_{seed}")
                )
                .collect()
            )

            if df_seed.height > 0:
                pred_dfs_per_seed.append(df_seed)

        # Không có seed nào có OOF cho behavior này → bỏ qua
        if not pred_dfs_per_seed:
            continue

        # Gộp các seed theo INDEX_COLS
        pred_merged = pl.concat(pred_dfs_per_seed, how="align")

        # Các cột score theo seed cho behavior này
        seed_cols = [c for c in pred_merged.columns if c.startswith(f"{behavior}_seed_")]

        # Mean score qua seed → 1 score ensemble cho behavior này
        behavior_scores = (
            pred_merged
            .with_columns(
                pl.mean_horizontal(pl.col(seed_cols)).alias(behavior)
            )
            .select(*INDEX_COLS, behavior)
        )

        prediction_dataframe_list.append(behavior_scores)

    # Bỏ qua nếu không có behavior nào có OOF cho nhóm này
    if not prediction_dataframe_list:
        continue

    # Kết hợp score của nhiều behavior theo chiều ngang (align theo INDEX_COLS)
    prediction_dataframe = pl.concat(prediction_dataframe_list, how="align")

    # Lấy tên các cột không phải INDEX_COLS (tức là tên behavior)
    cols = prediction_dataframe.select(pl.exclude(INDEX_COLS)).columns

    # Chọn behavior có score cao nhất mỗi frame
    prediction_labels_dataframe = (
        prediction_dataframe
        .with_columns(
            pl.struct(pl.exclude(INDEX_COLS))
            .map_elements(
                lambda row: (
                    "none"
                    if sum(row.values()) == 0
                    else cols[int(np.argmax(list(row.values())))]
                ),
                return_dtype=pl.String,
            )
            .alias("prediction")
        )
        .select(INDEX_COLS + ["prediction"])
    )

    # Gom các đoạn liên tiếp cùng prediction thành segments
    group_oof_prediction = (
        prediction_labels_dataframe
        # lấy các điểm biên khi prediction thay đổi
        .filter(pl.col("prediction") != pl.col("prediction").shift(1))
        # frame kế tiếp là stop_frame
        .with_columns(
            pl.col("video_frame").shift(-1).alias("stop_frame")
        )
        # bỏ "none"
        .filter(pl.col("prediction") != "none")
        .select(
            pl.col("video_id"),
            # agent_id: "mouse1", "mouse2", ...
            ("mouse" + pl.col("agent_mouse_id").cast(pl.Utf8)).alias("agent_id"),
            # target_id: "self" hoặc "mouseX"
            pl.when(pl.col("target_mouse_id") == -1)
            .then(pl.lit("self"))
            .otherwise("mouse" + pl.col("target_mouse_id").cast(pl.Utf8))
            .alias("target_id"),
            pl.col("prediction").alias("action"),
            pl.col("video_frame").alias("start_frame"),
            pl.col("stop_frame"),
        )
    )

    group_oof_predictions.append(group_oof_prediction)

# ====== robustify + lưu file OOF CSV dùng cho compute_validation_metrics ======
%run -i robustify.py

oof_predictions = pl.concat(group_oof_predictions, how="vertical")
oof_predictions = robustify(oof_predictions, train_dataframe, train_test="train")
oof_predictions.with_row_index("row_id").write_csv(WORKING_DIR / "oof_predictions.csv")

##Tính điểm (score) dựa trên dữ liệu kiểm chứng

In [ ]:

def compute_validation_metrics(submission, verbose=True):
    """Compute and display validation metrics for single vs pair behaviors."""
    # solution_df
    dataset = pl.read_csv(INPUT_DIR / "train.csv").to_pandas()

    solution = []
    for _, row in dataset.iterrows():
        lab_id = row["lab_id"]
        if lab_id.startswith("MABe22"):
            continue

        video_id = row["video_id"]
        path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            continue

        annot["lab_id"] = lab_id
        annot["video_id"] = video_id
        annot["behaviors_labeled"] = row["behaviors_labeled"]
        annot["target_id"] = np.where(
            annot.target_id != annot.agent_id, annot["target_id"].apply(lambda s: f"mouse{s}"), "self"
        )
        annot["agent_id"] = annot["agent_id"].apply(lambda s: f"mouse{s}")
        solution.append(annot)

    solution = pd.concat(solution)

    try:
        # Separate single and pair behaviors
        submission_single = submission[submission["target_id"] == "self"].copy()
        submission_pair = submission[submission["target_id"] != "self"].copy()

        # Filter solution to match submission videos
        solution_videos = set(submission["video_id"].unique())
        solution = solution[solution["video_id"].isin(solution_videos)]

        if len(solution) == 0:
            return

        # Compute overall F1 score
        overall_f1 = score(solution, submission, "row_id", beta=1.0)
        print(f"\n{'=' * 60}")
        print("PERFORMANCE METRICS")
        print(f"{'=' * 60}")
        print(f"Overall F1 Score: {overall_f1:.4f}")
        print(f"Total predictions: {len(submission)}")
        print(f"  - Single behaviors: {len(submission_single)}")
        print(f"  - Pair behaviors: {len(submission_pair)}")

        # Compute per-action F1 scores using existing scoring function
        solution_pl = pl.DataFrame(solution)
        submission_pl = pl.DataFrame(submission)

        # Add label_key and prediction_key
        solution_pl = solution_pl.with_columns(
            pl.concat_str(
                [
                    pl.col("video_id").cast(pl.Utf8),
                    pl.col("agent_id").cast(pl.Utf8),
                    pl.col("target_id").cast(pl.Utf8),
                    pl.col("action"),
                ],
                separator="_",
            ).alias("label_key"),
        )
        submission_pl = submission_pl.with_columns(
            pl.concat_str(
                [
                    pl.col("video_id").cast(pl.Utf8),
                    pl.col("agent_id").cast(pl.Utf8),
                    pl.col("target_id").cast(pl.Utf8),
                    pl.col("action"),
                ],
                separator="_",
            ).alias("prediction_key"),
        )

        # Group by action and compute metrics
        action_stats = defaultdict(lambda: {"single": {"count": 0, "f1": 0.0}, "pair": {"count": 0, "f1": 0.0}})

        for lab in solution_pl["lab_id"].unique():
            lab_solution = solution_pl.filter(pl.col("lab_id") == lab).clone()
            lab_videos = set(lab_solution["video_id"].unique())
            lab_submission = submission_pl.filter(pl.col("video_id").is_in(lab_videos)).clone()

            # Compute per-action F1 using same logic as single_lab_f1
            label_frames = defaultdict(set)
            prediction_frames = defaultdict(set)

            for row in lab_solution.to_dicts():
                label_frames[row["label_key"]].update(range(row["start_frame"], row["stop_frame"]))

            for row in lab_submission.to_dicts():
                key = row["prediction_key"]
                prediction_frames[key].update(range(row["start_frame"], row["stop_frame"]))

            for key in set(list(label_frames.keys()) + list(prediction_frames.keys())):
                action = key.split("_")[-1]
                mode = "single" if "self" in key else "pair"

                pred_frames = prediction_frames.get(key, set())
                label_frames_set = label_frames.get(key, set())

                tp = len(pred_frames & label_frames_set)
                fn = len(label_frames_set - pred_frames)
                fp = len(pred_frames - label_frames_set)

                if tp + fn + fp > 0:
                    f1 = (1 + 1**2) * tp / ((1 + 1**2) * tp + 1**2 * fn + fp)
                    action_stats[action][mode]["count"] += 1
                    action_stats[action][mode]["f1"] += f1

        # Print per-action summary
        print("\nPer-Action Performance Summary:")
        print(f"{'-' * 60}")
        print(f"{'Action':<20} {'Mode':<10} {'Count':<10} {'Avg F1':<10}")
        print(f"{'-' * 60}")

        for action in sorted(action_stats.keys()):
            for mode in ["single", "pair"]:
                stats = action_stats[action][mode]
                if stats["count"] > 0:
                    avg_f1 = stats["f1"] / stats["count"]
                    print(f"{action:<20} {mode:<10} {stats['count']:<10} {avg_f1:<10.4f}")

        # Summary by mode
        single_actions = [a for a in action_stats.keys() if action_stats[a]["single"]["count"] > 0]
        pair_actions = [a for a in action_stats.keys() if action_stats[a]["pair"]["count"] > 0]

        if single_actions:
            single_avg_f1 = np.mean(
                [
                    action_stats[a]["single"]["f1"] / action_stats[a]["single"]["count"]
                    for a in single_actions
                    if action_stats[a]["single"]["count"] > 0
                ]
            )
            print(f"\nSingle behaviors: {len(single_actions)} actions, Avg F1: {single_avg_f1:.4f}")

        if pair_actions:
            pair_avg_f1 = np.mean(
                [
                    action_stats[a]["pair"]["f1"] / action_stats[a]["pair"]["count"]
                    for a in pair_actions
                    if action_stats[a]["pair"]["count"] > 0
                ]
            )
            print(f"Pair behaviors: {len(pair_actions)} actions, Avg F1: {pair_avg_f1:.4f}")

        print(f"{'=' * 60}\n")

    except Exception as e:
        if verbose:
            error_msg = str(e)
            if len(error_msg) > 200:
                error_msg = error_msg[:200] + "..."
            print(f"\nWarning: Could not compute validation metrics: {error_msg}")
            if verbose:
                print(f"Traceback: {traceback.format_exc()[:300]}")

compute_validation_metrics(submission=pd.read_csv(WORKING_DIR / "oof_predictions.csv"))